In [ ]:
import geopandas as gpd
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np

## Latitude - north to south -> horizontal lines -> acts as y
## Longitude - east to west -> vertical lines -> acts as x

### Loadign the required data.

In [ ]:
shapefile_path = r"/home/vedanshi/Documents/DSML/IMD work/Narmada/NARMADA.shp"
gdf = gpd.read_file(shapefile_path)

In [ ]:
print(f"Current CRS: {gdf.crs}")

In [ ]:
gdf = gdf.to_crs(epsg=4326)

In [ ]:
nc_file = "/home/vedanshi/Documents/DSML/IMD work/Narmada/RF25_ind2005_rfp25.nc" # 2005s data is loaded.
dataset = Dataset(nc_file, 'r')

In [ ]:
path2='/home/vedanshi/Documents/DSML/IMD work/Narmada/RF25_ind2015_rfp25.nc' # 2015s data 
dataset1 = Dataset(path2, 'r')
path3='/home/vedanshi/Documents/DSML/IMD work/Narmada/RF25_ind2021_rfp25.nc' # 2021s data
dataset2 = Dataset(path3, 'r')

In [ ]:
print("All variable keys:")
print(dataset.variables.keys())#,'\n',dataset1.variables.keys(),'\n',dataset2.varaibles.keys())

In [ ]:
dataset1.variables.keys()

In [ ]:
dataset2.variables.keys()

In [ ]:
latitude = dataset.variables['LATITUDE'][:]
longitude = dataset.variables['LONGITUDE'][:]
precipitation = dataset.variables['RAINFALL'][:]
precipitation1 = dataset1.variables['RAINFALL'][:]
precipitation2 = dataset2.variables['RAINFALL'][:]
#latitude1 = dataset1.variables['LATITUDE'][:]
#longitude1 = dataset1.variables['LONGITUDE'][:]
#latitude2 = dataset2.variables['LATITUDE'][:]
#longitude2 = dataset2.variables['LONGITUDE'][:]

In [ ]:
latitude.ndim

In [ ]:
miny = 21.3
maxy = 23.8
minx = 72.6
maxx = 81.95

In [ ]:
# Create a mask for the bounding box
lat_mask = (latitude >= miny) & (latitude <= maxy)
lon_mask = (longitude >= minx) & (longitude <= maxx )

In [ ]:
# Crop the precipitation data to the bounding box
precipitation_cropped = precipitation[:, lat_mask, :][:, :, lon_mask]
precipitation_cropped1 = precipitation1[:, lat_mask, :][:, :, lon_mask]
precipitation_cropped2 = precipitation2[:, lat_mask, :][:, :, lon_mask]

In [ ]:
print(precipitation_cropped.shape==precipitation_cropped1.shape==precipitation_cropped2.shape)

In [ ]:
precipitation_day = precipitation_cropped[0]
precipitation_day1 = precipitation_cropped1[0]
precipitation_day2 = precipitation_cropped2[0]

In [ ]:
precipitation_day_flatten = precipitation_cropped[0].flatten()
precipitation_day1_faltten = precipitation_cropped1[0].flatten()
precipitation_day2_flatten = precipitation_cropped2[0].flatten()

In [ ]:
precip_day1_2005 = precipitation_day.filled(fill_value=0)
precip_day1_2015 = precipitation_day1.filled(fill_value=0)
precip_day1_2021 = precipitation_day2.filled(fill_value=0)

In [ ]:
precipitation={'2005':precip_day1_2005,'2015':precip_day1_2015,'2021':precip_day1_2021}

In [ ]:
precipitation['2015']

In [ ]:
precipitation['2021']

In [ ]:
precipitation['2005']

In [ ]:
colorbar_min = 0  # replace with your desired minimum value
colorbar_max = 10  # replace with your desired maximum value

fig, ax = plt.subplots(figsize=(10, 6))

# Create the heatmap on the Axes object `ax`
# Add vmin and vmax to set the range of the colorbar

c = ax.pcolormesh(longitude[lon_mask], latitude[lat_mask], precip_day1_2005, 
                shading='auto', vmin=colorbar_min, vmax=colorbar_max)
# Add the colorbar
fig.colorbar(c, ax=ax, label='1st jan Precipitation')
# Oerlay the basin boundary, make sure to specify the same Axes object `ax`
gdf.boundary.plot(ax=ax, color='red', linewidth=2, label='Basin')
# Ad the legend
ax.legend()
# Add labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('1st jan Precipitation Heatmap with Basin Boundary')
# Dsplay the plot
plt.show()

In [ ]:
colorbar_min = 0  # replace with your desired minimum value
colorbar_max = 10  # replace with your desired maximum value

fig, ax = plt.subplots(figsize=(10, 6))

# Create the heatmap on the Axes object `ax`
# Add vmin and vmax to set the range of the colorbar

c = ax.pcolormesh(longitude[lon_mask], latitude[lat_mask], precip_day1_2015, 
                shading='auto', vmin=colorbar_min, vmax=colorbar_max)
# Add the colorbar
fig.colorbar(c, ax=ax, label='1st Jan Precipitation')
# Oerlay the basin boundary, make sure to specify the same Axes object `ax`
gdf.boundary.plot(ax=ax, color='red', linewidth=2, label='Basin')
# Ad the legend
ax.legend()
# Add labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('1st Jan Precipitation Heatmap with Basin Boundary')
# Dsplay the plot
plt.show()

In [ ]:
colorbar_min = 0  # replace with your desired minimum value
colorbar_max = 10  # replace with your desired maximum value

fig, ax = plt.subplots(figsize=(10, 6))

# Create the heatmap on the Axes object `ax`
# Add vmin and vmax to set the range of the colorbar

c = ax.pcolormesh(longitude[lon_mask], latitude[lat_mask], precip_day1_2021, 
                shading='auto', vmin=colorbar_min, vmax=colorbar_max)
# Add the colorbar
fig.colorbar(c, ax=ax, label='1st Jan Precipitation')
# Oerlay the basin boundary, make sure to specify the same Axes object `ax`
gdf.boundary.plot(ax=ax, color='red', linewidth=2, label='Basin')
# Ad the legend
ax.legend()
# Add labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('1st Jan Precipitation Heatmap with Basin Boundary')
# Dsplay the plot
plt.show()

We have to find the weghted area average of the closest 3-4 points, and then assing that value to the cetroid, and add the values to get the area to the main centroid of the small region.

for that we'll first have to create the custom grid that is not present in this code.

### Creating custom grid.

In [ ]:
# gdf is the shape file for narmada.
# dataset is the rain data.
import geopandas as gpd 
import matplotlib as ma
import pandas as pd
import threading as thread
import fiona
import matplotlib as plt
import plotly.express as plt
import numpy as np

In [ ]:
bound_box=gdf.total_bounds
minx_grid,miny_grid,maxx_grid,maxy_grid=bound_box
bound_box

In [ ]:
from shapely.geometry import box
# Create a rectangle from these coordinates using shapely's box function
rectangle = gpd.GeoSeries([box(minx_grid, miny_grid, maxx_grid, maxy_grid)], crs=gdf.crs)
# Print the rectangle coordinates
print(f"Bounding Box Coordinates: {bound_box}")
print(f"Rectangle Geometry: {rectangle}")

In [ ]:
ax=gdf.plot(color='blue',edgecolor='k')
rectangle.plot(ax=ax, color='none', edgecolor='red') # I don't understand why is this sometiems shows an error.

In [ ]:
#Increase the rectangle size by 15 km in both height and width
minx_new = minx_grid - 0.125
miny_new = miny_grid - 0.125
maxx_new = maxx_grid + 0.125
maxy_new = maxy_grid + 0.125

In [ ]:
new_bounding_box = [minx_new, miny_new, maxx_new, maxy_new]
rectangle_new = gpd.GeoSeries([box(minx_new, miny_new, maxx_new, maxy_new)], crs=gdf.crs)
# Print the new bounding box coordinates as integers
print(f"New Bounding Box Coordinates: {list(map(float, new_bounding_box))}")

# Calculate and print the dimensions of the new rectangle as integers
width_new = maxx_new - minx_new
height_new = maxy_new - miny_new
print(f"New Rectangle Dimensions: Width = {float(width_new)}, Height = {float(height_new)}")

In [ ]:
ax = gdf.plot(color='purple', edgecolor='k')
rectangle_new.plot(ax=ax, color='none', edgecolor='blue')

In [ ]:
# using minx and maxx, i.e the boundary poits for the grid.
# initialize an empty list to store x-values for the custom grid
x_values = []
# generate x-values
current_x = minx
increment = 0.25 #this is an inceremnet of 0.25
while current_x <= maxx:
    x_values.append(current_x)  
    current_x += increment
len(x_values)

In [ ]:
# using miny and maxy, i.e the boundary poits for the grid.
# initialize an empty list to store y-values for the custom grid
y_values = []
# generate y-values
current_y = miny
increment = 0.25 #this is an inceremnt of 0.25
while current_y <= maxy:
    y_values.append(current_y)  
    current_y += increment
len(y_values)

In [ ]:
x_values = np.array(x_values, dtype=float) # x_values are the longitudes and y_values are the latitudes.
y_values = np.array(y_values, dtype=float)

In [ ]:
# function to calculate grid parameters
def calculate_grid_parameters(coord):
    d = np.diff(coord)[0] / 2
    cell_boundaries = np.concatenate([[coord[0] - d], coord + d])
    return cell_boundaries

In [ ]:
# function to create grid cell taking arguments from x_values and y_values
def initialize_and_populate_grid(x_values, y_values):
    ngrid_cell = len(x_values) * len(y_values)
    grid_cell = [{} for _ in range(ngrid_cell)]
    
    dy = y_values[1] - y_values[0]
    k = 0
    
    loncell = calculate_grid_parameters(x_values)
    latcell = calculate_grid_parameters(y_values)
    
    for i, lon in enumerate(x_values):
        for j, lat in enumerate(y_values):
            x_coords = [round(loncell[i], 2), round(loncell[i], 2), round(loncell[i+1], 2), round(loncell[i+1], 2), round(loncell[i], 2)]
            y_coords = [round(latcell[j], 2), round(latcell[j+1], 2), round(latcell[j+1], 2), round(latcell[j], 2), round(latcell[j], 2)] if dy > 0 else \
                       [round(latcell[j+1], 2), round(latcell[j], 2), round(latcell[j], 2), round(latcell[j+1], 2), round(latcell[j+1], 2)]
            
            grid_cell[k] = {'X': x_coords, 'Y': y_coords, 'Xc': round(lon, 2), 'Yc': round(lat, 2)}
            k += 1
    return grid_cell

In [ ]:
if __name__ == "__main__":
    grid_cell = initialize_and_populate_grid(x_values, y_values)

In [ ]:
grid_cell

In [ ]:
len(grid_cell)==38*11

In [ ]:
# creating a geodataframe for grid cells as polygons
from shapely.geometry import Polygon
import geopandas as gpd

grid_cells = []

for cell in grid_cell:
    polygon = Polygon(list(zip(cell['X'], cell['Y'])))
    grid_cells.append(polygon)

grid_gdf = gpd.GeoDataFrame({'geometry': grid_cells}, crs=gdf.crs)

print(grid_gdf.head())

In [ ]:
grid_gdf.iloc[35]

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
grid_gdf.boundary.plot(ax=ax, color='blue',label='grid cells')
gdf.boundary.plot(ax=ax, color='red',label='basin')
plt.legend()
plt.show()

## We now have to find the total precipitation of the grid, and then find the area average and assign it to Narmada on its centroid

### The process of finding weighted average begins

In [ ]:
A=0.25**2 #area of one grid cell.

In [ ]:
intersecting_cells = gpd.sjoin(grid_gdf, gdf, how="inner", predicate="intersects")
num_intersecting_cells = len(intersecting_cells)
print(f"Number of intersecting cells: {num_intersecting_cells}")


In [ ]:
# Calculate the intersection area
intersecting_cells['intersection_area'] = intersecting_cells.apply(lambda row: row['geometry'].intersection(gdf.loc[row['index_right'], 'geometry']).area, axis=1)

In [ ]:
gdf

In [ ]:
grid_gdf

In [ ]:
completely_inside = grid_gdf['geometry'].within(gdf.iloc[0]['geometry'])
completely_inside_indices = [index for index, inside in enumerate(grid_gdf['geometry'].within(gdf.iloc[0]['geometry'])) if inside]
completely_inside_indices
#len(completely_inside_indices)

In [ ]:
#plotting one of the cell, that we found out to be inside the basin, and completely inside
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
grid_gdf.boundary.plot(ax=ax, color='blue', label='grid cells')
grid_polygon = grid_gdf.iloc[69]['geometry']
ax.fill(*grid_polygon.exterior.xy, color='black', alpha=0.9, label='69th grid cell')
gdf.boundary.plot(ax=ax, color='red', label='basin')
plt.legend()
plt.show()

In [ ]:
#plotting one of the cell, that we found out to be inside the basin, and completely inside
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
grid_gdf.boundary.plot(ax=ax, color='blue', label='grid cells')
grid_polygon = grid_gdf.iloc[36]['geometry']
ax.fill(*grid_polygon.exterior.xy, color='black', alpha=0.9, label='36th grid cell')
gdf.boundary.plot(ax=ax, color='red', label='basin')
plt.legend()
plt.show()

In [ ]:
grid_cell # we can directly use grid_cell, as its easier to extract

In [ ]:
intersecting_cells.columns

In [ ]:
intersecting_cells['intersection_type'] = 0  # Initialize all values to 0

for index in completely_inside_indices:
    intersecting_cells.loc[index, 'intersection_type'] = 1

intersecting_cells['intersection_type']


In [ ]:
l=intersecting_cells.index
print(l)

In [ ]:
intersecting_cells['intersection_area']=pd.NA
for i in l:
    if intersecting_cells['intersection_type'][i]==1:
        intersecting_cells['intersection_area']=A
    else:
        intersecting_cells.loc[i, 'intersection_area'] = intersecting_cells.loc[i, 'geometry'].intersection(gdf.loc[intersecting_cells.loc[i, 'index_right'], 'geometry']).area

In [ ]:
grid_cell[12]

In [ ]:
rain=[]
for i in l:
    rain.append([grid_cell[i]['Xc'],grid_cell[i]['Yc']])
rain_coords=pd.DataFrame(rain) #Centroids.

In [ ]:
rain_coords

In [ ]:
# Extracting the lat and lon of the basin for which we will interpolate, the rain at the centorids.
lat_req=[]
for i in dataset['LATITUDE']:
    if miny_grid<i<maxy_grid:
        lat_req.append(i)
lon_req=[]
for j in dataset['LONGITUDE']:
    if minx_grid<j<maxx_grid:
        lon_req.append(j)

In [ ]:
lon,lat=np.meshgrid(lon_req,lat_req)

In [ ]:
prcp_points= np.column_stack((lon.ravel(), lat.ravel()))

In [ ]:
def interpolate(coords,year,method_used):
    #entered coords must be upto accurcay of only 1 digit after the decimal.
    import scipy.interpolate as sci
    if year==2005:
        prcp_values_flat=precipitation['2005'].flatten()
    elif year==2015:
         prcp_values_flat=precipitation['2015'].flatten()
    elif year==2021:
         prcp_values_flat=precipitation['2021'].flatten()
    if method_used=='linear' or method_used=='bilinear':
        if method_used=='bilinear':
            return sci.griddata(prcp_points,prcp_values_flat,coords,method='nearest')
        else:
            return sci.griddata(prcp_points,prcp_values_flat,coords,method='linear')
    else :
            inter=sci.NearestNDInterpolator(prcp_points,prcp_values_flat,rescale=False,tree_options=None)
            return inter(coords)

In [ ]:
def accuracy(y_true,y_interpolated):
    # Calculate MAE
    mae = np.mean(np.abs(y_interpolated - y_true))
    # Calculate MSE
    mse = np.mean((y_interpolated - y_true)**2)
    # Calculate RMSE
    rmse = np.sqrt(mse)
    # Calculate R^2 score
    mean_true = np.mean(y_true)
    r2 = 1 - np.sum((y_true - y_interpolated)**2) / np.sum((y_true - mean_true)**2)
    print("MAE:", mae)
    print("MSE:", mse)
    print("RMSE:", rmse)
    print("R^2 Score:", r2)


In [ ]:
print(prcp_points.shape,precipitation['2005'].flatten().shape)

In [ ]:
interpolate(rain_coords,2015,'linear')

In [ ]:
interpolate(rain_coords,2015,'bilinear')

In [ ]:
interpolate(rain_coords,2015,'knn')

In [ ]:
intersecting_cells['prcp_2005']=interpolate(rain_coords,2005,'bilinear')
intersecting_cells['prcp_2015']=interpolate(rain_coords,2015,'bilinear')
intersecting_cells['prcp_2021']=interpolate(rain_coords,2021,'bilinear')

In [ ]:
intersecting_cells['intersection_area'].tail(30)

In [ ]:
# Performing the actual process.
def calculate_rain(df,year):
    prcp=[]
    for i,j in zip(df['intersection_area'], df[f'prcp_{year}']):
        const=j/A
        prcp.append(const*i)
    return prcp

we now have to map these values to the respective centroid coords.

In [ ]:
# These are centroids, to which we are mapping the rain on the centroid coords.
rain_coords['rain_2005']=calculate_rain(intersecting_cells,2005)
rain_coords['rain_2015']=calculate_rain(intersecting_cells,2015)
rain_coords['rain_2021']=calculate_rain(intersecting_cells,2021)
rain_coords.rename(columns={0:'longitude centroids'}, inplace=True)
rain_coords.rename(columns={1:'latitude centroids'}, inplace=True)
rain_coords

In [ ]:
count = sum(1 for area in intersecting_cells['intersection_area'] if area == A) 
count

We will hvae to insert the prcp data in the grid_cell

In [ ]:
main_centorid_lon=0.0
main_centorid_lat=0.0
sum_lon=0.0 # x coordinate
sum_lat=0.0 # y coordinate
for i in grid_cell:
    sum_lon+=i['Xc']
    sum_lat+=i['Yc']
main_centorid_lon=sum_lon/len(grid_cell)
main_centorid_lat=sum_lat/len(grid_cell)

In [ ]:
# We now have to just get the weighted average on this coordinate
print(main_centorid_lon)
print(main_centorid_lat)

In [ ]:
import matplotlib.pyplot as plt
# Assuming you already have grid_gdf and gdf defined somewhere in your code
fig, ax = plt.subplots()
# Plot grid cells boundary
grid_gdf.boundary.plot(ax=ax, color='blue', label='grid cells')
# Plot basin boundary
gdf.boundary.plot(ax=ax, color='red', label='basin')
# Plot main centroid
ax.scatter(main_centorid_lon, main_centorid_lat, color='black', label='main centroid')
plt.legend()
plt.show()

### Finding the weighted average for 2005, 2015 & 2021

In [ ]:
#temp=[]
#for i in range(196):
#    temp.append([rain_coords['longitude centroids'][i],rain_coords['latitude centroids'][i]])
#print(temp)
#print(rain_coords['rain_2005'].shape)

In [ ]:
main_cent_value={'2005':rain_coords['rain_2005'].sum(),'2015':rain_coords['rain_2015'].sum(),'2021':rain_coords['rain_2021'].sum()}

In [ ]:
#import scipy.interpolate as sci
#cords=(main_centorid_lon,main_centorid_lat)
#weighted_avg={'2005':sci.griddata(temp,rain_coords['rain_2005'],cords,method='linear'),'2015':sci.griddata(temp,rain_coords['rain_2015'],cords,method='linear'),'2021':sci.griddata(temp,rain_coords['rain_2021'],cords,method='linear')}

In [ ]:
# for 2005
fig, ax = plt.subplots()
# Plot grid cells boundary
grid_gdf.boundary.plot(ax=ax, color='blue', label='grid cells')
# Plot basin boundary
gdf.boundary.plot(ax=ax, color='red', label='basin')
# Plot main centroid
ax.scatter(main_centorid_lon, main_centorid_lat, color='black', label='main centroid')
# Add label or annotation for the main centroid point
main_centroid_value=main_cent_value['2005']
text_offset = 0.5
ax.text(main_centorid_lon, main_centorid_lat-text_offset, f'{main_centroid_value}', color='black', fontsize=13, ha='center')

plt.legend()
plt.show()


In [ ]:
# for 2015
fig, ax = plt.subplots()
# Plot grid cells boundary
grid_gdf.boundary.plot(ax=ax, color='blue', label='grid cells')
# Plot basin boundary
gdf.boundary.plot(ax=ax, color='red', label='basin')
# Plot main centroid
ax.scatter(main_centorid_lon, main_centorid_lat, color='black', label='main centroid')
# Add label or annotation for the main centroid point
main_centroid_value=main_cent_value['2015']
text_offset = 0.5
ax.text(main_centorid_lon, main_centorid_lat-text_offset, f'{main_centroid_value}', color='black', fontsize=13, ha='center')
plt.legend()
plt.show()

In [ ]:
# for 2021
fig, ax = plt.subplots()
# Plot grid cells boundary
grid_gdf.boundary.plot(ax=ax, color='blue', label='grid cells')
# Plot basin boundary
gdf.boundary.plot(ax=ax, color='red', label='basin')
# Plot main centroid
ax.scatter(main_centorid_lon, main_centorid_lat, color='black', label='main centroid')
# Add label or annotation for the main centroid point
main_centroid_value=main_cent_value['2021']
text_offset = 0.5
ax.text(main_centorid_lon, main_centorid_lat-text_offset, f'{main_centroid_value}', color='black', fontsize=13, ha='center')

plt.legend()
plt.show()
